# The Rosenbluth-Fokker-Planck equation

- Import all relevant modules

In [1]:
import torch
from pyapes.geometry import Cylinder
from pyapes.mesh import Mesh
from pyrfp.simulators.rfp import RFP_RZ
from pyrfp.training_data import RosenbluthPotentials_RZ
from pymytools.constants import PI


- Set some parameters to be used later

In [2]:
grids = [[32, 64], [64, 128], [128, 256], [256, 512]]

- Solver accuracy at different grids

In [8]:
for g in grids:
    mesh = Mesh(Cylinder[0:5, -5:5], None, g, device="cpu")
    dist = 1.0 / (2.0 * PI) ** 1.5 * torch.exp(-0.5 * (mesh.R**2 + mesh.Z**2))

    density = torch.sum(dist * 2.0 * PI * mesh.dx[0] * mesh.dx[1] * mesh.R)
    dist /= density
    
    rp_rz = RosenbluthPotentials_RZ(mesh, solver_config={
            "method": "bicgstab",
            "tol": 1e-6,
            "max_it": 1000,
            "report": False,
        })
    
    res_sol = rp_rz.from_pdf(dist, disp=False)
    res_analytic = rp_rz.from_analytic(dist, disp=False, low_mem=True)
    
    assert res_sol["timer"] is not None
    assert res_analytic["timer"] is not None
    
    assert res_sol["pots"] is not None
    assert res_analytic["pots"] is not None
    
    H_sol = res_sol["pots"]["H"]
    G_sol = res_sol["pots"]["G"]
    
    H_analytic = res_analytic["pots"]["H"]
    G_analytic = res_analytic["pots"]["G"]
    
    H_norm = torch.linalg.norm((H_sol - H_analytic))
    G_norm = torch.linalg.norm((G_sol - G_analytic))

    print(f"Grid: {g}, H_norm: {H_norm:.2e}, G_norm: {G_norm:.2e}")
    res_sol["timer"].display()
    
    rp_rz.timer.reset()



Grid: [32, 64], H_norm: 3.06e-01, G_norm: 1.52e-01


             Timer Report             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│  H_bc  │     0.01370      │   1    │
│ H_pot  │     0.04337      │   1    │
│  G_bc  │     0.01055      │   1    │
│ G_pot  │     0.04430      │   1    │
│ aH_pot │     0.11900      │   1    │
│ aG_pot │     0.11784      │   1    │
└────────┴──────────────────┴────────┘

Grid: [64, 128], H_norm: 1.69e-01, G_norm: 7.39e-02


             Timer Report             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│  H_bc  │     0.06455      │   1    │
│ H_pot  │     0.18856      │   1    │
│  G_bc  │     0.05779      │   1    │
│ G_pot  │     0.19236      │   1    │
│ aH_pot │     1.23168      │   1    │
│ aG_pot │     1.21660      │   1    │
└────────┴──────────────────┴────────┘

Grid: [128, 256], H_norm: 9.27e-02, G_norm: 3.63e-02


             Timer Report             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│  H_bc  │     0.44978      │   1    │
│ H_pot  │     1.15529      │   1    │
│  G_bc  │     0.43746      │   1    │
│ G_pot  │     0.84242      │   1    │
│ aH_pot │     17.33314     │   1    │
│ aG_pot │     16.68290     │   1    │
└────────┴──────────────────┴────────┘

Grid: [256, 512], H_norm: 5.05e-02, G_norm: 1.80e-02


             Timer Report             
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃  Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│  H_bc  │     8.29060      │   1    │
│ H_pot  │     7.54713      │   1    │
│  G_bc  │     6.20875      │   1    │
│ G_pot  │     7.08256      │   1    │
│ aH_pot │    243.29937     │   1    │
│ aG_pot │    239.40632     │   1    │
└────────┴──────────────────┴────────┘

- Computational cost with different grid sized

In [9]:

for g in grids:
    mesh = Mesh(Cylinder[0:5, -5:5], None, g, device="cpu")

    dist = 1.0 / (2.0 * PI) ** 1.5 * torch.exp(-0.5 * (mesh.R**2 + mesh.Z**2))

    density = torch.sum(dist * 2.0 * PI * mesh.dx[0] * mesh.dx[1] * mesh.R)
    dist /= density

    rfp_rz = RFP_RZ(mesh, dist, 0.0001, 10)
    rfp_rz.run(no_update=True)
    print(f"Tested grid: {g} on CPU.")
    rfp_rz.timer.display()
    rfp_rz.timer.reset()
    
# Repeat if cuda is available
if torch.cuda.is_available():
    for g in grids:
        mesh = Mesh(Cylinder[0:5, -5:5], None, g, device="cuda")

        dist = 1.0 / (2.0 * PI) ** 1.5 * torch.exp(-0.5 * (mesh.R**2 + mesh.Z**2))

        density = torch.sum(dist * 2.0 * PI * mesh.dx[0] * mesh.dx[1] * mesh.R)
        dist /= density

        rfp_rz = RFP_RZ(mesh, dist, 0.0001, 10)
        rfp_rz.run(no_update=True)
        print(f"Tested grid: {g} on CUDA.")
        rfp_rz.timer.display()
        rfp_rz.timer.reset()


Tested grid: [32, 64]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     0.08483      │   10   │
│ H_sol │     0.37782      │   10   │
│ G_bc  │     0.08501      │   10   │
│ G_sol │     0.39632      │   10   │
└───────┴──────────────────┴────────┘

Tested grid: [64, 128]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     0.58919      │   10   │
│ H_sol │     1.41298      │   10   │
│ G_bc  │     0.57922      │   10   │
│ G_sol │     1.62845      │   10   │
└───────┴──────────────────┴────────┘

Tested grid: [128, 256]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     4.18639      │   10   │
│ H_sol │     8.58235      │   10   │
│ G_bc  │     4.10070      │   10   │
│ G_sol │     8.45804      │   10   │
└───────┴──────────────────┴────────┘

Tested grid: [256, 512]


            Timer Report             
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Name  ┃ Elapsed time [s] ┃ #Calls ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ H_bc  │     51.00295     │   10   │
│ H_sol │     72.18819     │   10   │
│ G_bc  │     44.70981     │   10   │
│ G_sol │     66.93341     │   10   │
└───────┴──────────────────┴────────┘